# Excel - анализ 

Вопросы по смыслу полей.
- REGION_NM (регион РФ)
Регион по регистрации, пребыванию, почте?
- REGION_NM (регион РФ)
Регион ЦЕНТРАЛЬНЫЙ ОФИС - что это?
- LOAN_MAX_DLQ (номер максимальной просрочки, допущенной клиентом)
Что это?


Странные доходы PERSONAL_INCOME 
- 1950 руб. 23 года, пенсия. Возможно, инвалидность. Оставить как есть.
- 24 руб. Видимо, опечатка. Импутировать по среднему, или по среднему для 28-летних, военнослужащих по контракту.
- 6299,19 руб. Единственный доход с копейками. Возможно пенсия. Оставить как есть.

Наличие в заявке рабочего телефона GEN_PHONE_FL при отсутствии работы SOCSTATUS_WORK_FL.
- Доход похож на пенсию в 2011 году. Возможно, указано последнее место работы либо обман. Удалить или оставить до выяснения.

Наличие работы (GEN_INDUSTRY, GEN_TITLE) при отсутствии работы SOCSTATUS_WORK_FL. 
- Практически совпадает с "Наличие в заявке рабочего телефона при отсутствии работы". Доход похож на пенсию в 2011 году. Возможно, указано последнее место работы либо обман. Удалить или оставить до выяснения.

Странное (очень большое) время работы в месяцах WORK_TIME. 
- Исходя из максимального возраста в 67 лет, вряд ли стаж будет более 720 месяцев (60 лет х 12 месяцев) на последнем месте. Может быть, конечно, несколько совместительств. Обнулить всё, что больше 720, затем заменить средним.

Наличие пропусков в месте взятия кредита TP_PROVINCE при наличии кредита CREDIT.
- Клиент сказал сколько взял (см. CREDIT), но не сказал где? Можно предположить, что по месту фактического пребывания FACT_ADDRESS_PROVINCE. Соответственно, заменить пропуски 

REGION_NM - один пропуск, опечатка. 
- Заменить на ЗАПАДНО-СИБИРСКИЙ, т.к. Новосибирская область. Заменил в исходном csv файле

In [45]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [46]:
os.chdir('D:\\ML\\Gewissta\\My course\\Credit OTP\\')

In [47]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_info_columns',150)

In [48]:
df = pd.read_csv('Credit_OTP.csv', sep = ';', encoding='Windows-1251 ', decimal = ',')

In [49]:
df.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000.0,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000.00,6,8650.0,0,1,220,18.0,0,0,1,1,1,6,2,1,1580.000000,1580.0,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650.00,6,4000.0,0,1,137,97.0,1,0,1,1,1,6,1,1,4020.000000,4020.0,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000.0,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126.00,12,4000.0,0,1,251,84.0,0,0,1,2,1,11,0,0,0.000000,0.0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000.0,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,8491.82,6,5000.0,0,0,36,168.0,1,1,1,1,1,6,3,1,1589.923333,1590.0,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990.00,12,4000.0,0,1,83,101.0,1,0,1,2,1,16,2,1,1152.150000,2230.0,NaN


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AGREEMENT_RK                15223 non-null  int64  
 1   TARGET                      15223 non-null  int64  
 2   AGE                         15223 non-null  int64  
 3   SOCSTATUS_WORK_FL           15223 non-null  int64  
 4   SOCSTATUS_PENS_FL           15223 non-null  int64  
 5   GENDER                      15223 non-null  int64  
 6   CHILD_TOTAL                 15223 non-null  int64  
 7   DEPENDANTS                  15223 non-null  int64  
 8   EDUCATION                   15223 non-null  object 
 9   MARITAL_STATUS              15223 non-null  object 
 10  GEN_INDUSTRY                13856 non-null  object 
 11  GEN_TITLE                   13856 non-null  object 
 12  ORG_TP_STATE                13856 non-null  object 
 13  ORG_TP_FCAPITAL             138

In [51]:
# странно, почему PREVIOUS_CARD_NUM_UTILIZED float, а не int

In [52]:
df.isnull().sum()

AGREEMENT_RK                      0
TARGET                            0
AGE                               0
SOCSTATUS_WORK_FL                 0
SOCSTATUS_PENS_FL                 0
GENDER                            0
CHILD_TOTAL                       0
DEPENDANTS                        0
EDUCATION                         0
MARITAL_STATUS                    0
GEN_INDUSTRY                   1367
GEN_TITLE                      1367
ORG_TP_STATE                   1367
ORG_TP_FCAPITAL                1365
JOB_DIR                        1367
FAMILY_INCOME                     0
PERSONAL_INCOME                   0
REG_ADDRESS_PROVINCE              0
FACT_ADDRESS_PROVINCE             0
POSTAL_ADDRESS_PROVINCE           0
TP_PROVINCE                     295
REGION_NM                         0
REG_FACT_FL                       0
FACT_POST_FL                      0
REG_POST_FL                       0
REG_FACT_POST_FL                  0
REG_FACT_POST_TP_FL               0
FL_PRESENCE_FL              

До разбиения
- "1365-1368" - пенсионеры. Не импутировать. Сделать новые категории "Не указано"
- Первоначально удалить в ORG_TP_FCAPITAL значения "Без участия" для 'SOCSTATUS_PENS_FL = 1 (Пенсионер) и GEN_INDUSTRY = NaN. Опечатка
- Пропуски "1365-1368" заменить на "Не указано"
- Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 1 (Пенсионер) заменить на "0"
- Пропуски TP_PROVINCE (область торговой точки, где клиент брал последний кредит) - заменить на значение области фактического пребывания FACT_ADDRESS_PROVINCE
- Пропуски PREVIOUS_CARD_NUM_UTILIZED (количество уже утилизированных карт (если пусто - 0))заменить на "0"  

После разбиения
- WORK_TIME. Обнулить всё, что больше 720, затем заменить средним после разбиения(см. выше)
- Из материалов "Модуль1". WORK_TIME > 720 заменить медианой после разбиения

- Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 0 (Не пенсионер) заменить средним после разбиения
- Из материалов "Модуль1". Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 0 (Не пенсионер) заменить медианой после разбиения

In [53]:
#Посмотреть, нет ли опечаток, странных значений в категориальных переменных
cat_columns = df.select_dtypes(include = ['object']).columns.tolist()
for col in cat_columns:
    print(col, df[col].unique())

EDUCATION ['Среднее специальное' 'Среднее' 'Неполное среднее' 'Высшее'
 'Неоконченное высшее' 'Два и более высших образования' 'Ученая степень']
MARITAL_STATUS ['Состою в браке' 'Гражданский брак' 'Разведен(а)' 'Не состоял в браке'
 'Вдовец/Вдова']
GEN_INDUSTRY ['Торговля' 'Информационные технологии' 'Образование'
 'Государственная служба' 'Другие сферы' 'Сельское хозяйство'
 'Здравоохранение' 'Металлургия/Промышленность/Машиностроение' nan
 'Коммунальное хоз-во/Дорожные службы' 'Строительство' 'Транспорт'
 'Банк/Финансы' 'Ресторанный бизнес/Общественное питание' 'Страхование'
 'Нефтегазовая промышленность' 'СМИ/Реклама/PR-агенства' 'Энергетика'
 'Салоны красоты и здоровья' 'ЧОП/Детективная д-ть'
 'Развлечения/Искусство' 'Наука' 'Химия/Парфюмерия/Фармацевтика'
 'Сборочные производства' 'Туризм'
 'Юридические услуги/нотариальные услуги' 'Маркетинг' 'Подбор персонала'
 'Информационные услуги' 'Недвижимость' 'Управляющая компания' 'Логистика']
GEN_TITLE ['Рабочий' 'Специалист' 'Руководите

In [54]:
#Удалить дубликаты
df.drop_duplicates(subset = None, keep = 'first', inplace=True)

In [55]:
len(df)

15223

In [56]:
#Дубликатов нет

In [57]:
# Сделать замены пропусков на "Не указано"

In [58]:
df['ORG_TP_FCAPITAL'] = np.where((df['ORG_TP_FCAPITAL'] == "Без участия")&(df['GEN_INDUSTRY'].isnull())&(df['SOCSTATUS_PENS_FL'] == 1), 
                                                                "Не указано", df['ORG_TP_FCAPITAL'])

In [59]:
column_list = ['GEN_INDUSTRY', 'GEN_TITLE', 'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR']

In [60]:
for col in column_list:
    df[col] = np.where((df[col].isnull())&(df['SOCSTATUS_PENS_FL'] == 1), "Не указано",df[col])

In [61]:
for col in column_list:
    df[col] = np.where((df[col].isnull())&(df['SOCSTATUS_PENS_FL'] == 0), df[col].mode(),df[col])

In [62]:
# поиск редких категорий
for col in cat_columns:
    print(df[col].value_counts(dropna = False))

Среднее специальное               6518
Среднее                           4679
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: EDUCATION, dtype: int64
Состою в браке        9416
Не состоял в браке    3622
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: MARITAL_STATUS, dtype: int64
Торговля                                     2386
Другие сферы                                 1709
Не указано                                   1366
Металлургия/Промышленность/Машиностроение    1356
Государственная служба                       1286
Здравоохранение                              1177
Образование                                   998
Транспорт                                     787
Сельское хозяйство                            702
Строительство                                 574
Коммунальное хоз-во/Дорожны

In [63]:
#замена редких категорий 
df.loc[df['EDUCATION'] == 'Ученая степень', 'EDUCATION'] = 'Два и более высших образования'
df.loc[df['GEN_INDUSTRY'] == 'Маркетинг', 'GEN_INDUSTRY'] = 'СМИ/Реклама/PR-агенства'
df.loc[df['JOB_DIR'] == 'Реклама и маркетинг', 'JOB_DIR'] = 'Участие в основ. деятельности'

In [64]:
#Пропуски WORK_TIME для 'SOCSTATUS_PENS_FL = 1 (Пенсионер) заменить на "0"
df['WORK_TIME'] = np.where((df['WORK_TIME'].isnull())&(df['SOCSTATUS_PENS_FL'] == 1), 0, df['WORK_TIME'])

In [65]:
#Пропуски TP_PROVINCE (область торговой точки, где клиент брал последний кредит) - заменить на 
#значение области фактического пребывания FACT_ADDRESS_PROVINCE
df['TP_PROVINCE'] = np.where(df['TP_PROVINCE'].isnull(), df['FACT_ADDRESS_PROVINCE'], df['TP_PROVINCE'])

In [66]:
#Пропуски PREVIOUS_CARD_NUM_UTILIZED (количество уже утилизированных карт (если пусто - 0))заменить на "0"
#Преобразовать в intdf['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(df['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, df['PREVIOUS_CARD_NUM_UTILIZED'])
df['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(df['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, df['PREVIOUS_CARD_NUM_UTILIZED'])
df['PREVIOUS_CARD_NUM_UTILIZED'] = df['PREVIOUS_CARD_NUM_UTILIZED'].astype('int64')

In [67]:
#Кто такой не пенсионер и без рабочего времени.  
df[(df['WORK_TIME'].isnull()) & (df['SOCSTATUS_PENS_FL'] == 0)]

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
7963,64469089,0,24,1,0,0,0,0,Среднее специальное,Не состоял в браке,Торговля,Специалист,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,10000.0,Курская область,Курская область,Курская область,Курская область,ЦЕНТРАЛЬНЫЙ 1,1,1,1,1,1,0,0,0,0,0,0,0,5309.0,10,590.0,0,0,85,NaN,1,0,0,1,0,5,1,1,540.0,540.0,0


In [68]:
# Скорее всего, опечатка, т.к. указано "Частная компания". 
#Можно импутировать всего одно значение WORK_TIME и до разбиения, например, средним для 24-летних
df.loc[7963,'WORK_TIME'] = df[(df['AGE'] == 24)]['WORK_TIME'].mean()

In [69]:
# Проверка импутаций
df.isnull().sum()

AGREEMENT_RK                  0
TARGET                        0
AGE                           0
SOCSTATUS_WORK_FL             0
SOCSTATUS_PENS_FL             0
GENDER                        0
CHILD_TOTAL                   0
DEPENDANTS                    0
EDUCATION                     0
MARITAL_STATUS                0
GEN_INDUSTRY                  0
GEN_TITLE                     0
ORG_TP_STATE                  0
ORG_TP_FCAPITAL               0
JOB_DIR                       0
FAMILY_INCOME                 0
PERSONAL_INCOME               0
REG_ADDRESS_PROVINCE          0
FACT_ADDRESS_PROVINCE         0
POSTAL_ADDRESS_PROVINCE       0
TP_PROVINCE                   0
REGION_NM                     0
REG_FACT_FL                   0
FACT_POST_FL                  0
REG_POST_FL                   0
REG_FACT_POST_FL              0
REG_FACT_POST_TP_FL           0
FL_PRESENCE_FL                0
OWN_AUTO                      0
AUTO_RUS_FL                   0
HS_PRESENCE_FL                0
COT_PRES

In [70]:
# Преобразование object -> str для более простой работы с пропусками. Не понадобилось.
##df.astype({'GEN_INDUSTRY':'str', 'GEN_TITLE':'str', 'ORG_TP_STATE':'str', 'ORG_TP_FCAPITAL':'str', 'JOB_DIR':'str'}).dtypes

In [71]:
##df.describe()

In [72]:
#на всякий случай сохраним перед удалением столбцов
df_copy = df.copy()

In [73]:
#dummi-кодирование (без которого никак при LogisticRegression) даст очень большое количество предиктов
#Можно предположить, что удалив пребывание, регистрацию, почту и место выдачи, не получим серьёзного искажения.
#Достаточно региона.
df_short = df.drop(columns = ['REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE'])

In [74]:
X_train, X_test, y_train, y_test = train_test_split(df_short.drop('TARGET', axis=1), 
                                                df_short['TARGET'], 
                                                test_size=.3, 
                                                stratify=df_short['TARGET'], 
                                                random_state=155)

In [75]:
#WORK_TIME > 720 заменить медианой
X_train['WORK_TIME'] = np.where(X_train['WORK_TIME'] > 720, X_train['WORK_TIME'].median(), X_train['WORK_TIME'])
X_test['WORK_TIME'] = np.where(X_test['WORK_TIME'] > 720, X_test['WORK_TIME'].median(), X_test['WORK_TIME'])

In [76]:
#Bining class - из "Модуль 1"
class CustomBinning(BaseEstimator, TransformerMixin):
    def __init__(self, bins = [-np.inf, 30, 40, 50, 60, np.inf]):
        self.bins = bins
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        bins = np.array(self.bins)
        X_replaced = np.digitize(X, bins)
        return X_replaced

In [77]:
# Столбцы для различной обработки
cat_columns = X_train.select_dtypes(include = ['object']).columns
num_columns = X_train.select_dtypes(exclude = ['object']).columns
num_columns_for_bins = ['AGE']

In [78]:
cat_pipe = Pipeline([('ohe', OneHotEncoder(sparse = 'False', handle_unknown = 'ignore'))])
num_pipe = Pipeline([('scaler', StandardScaler())])
num_bins_pipe = Pipeline([('bin', CustomBinning()), 
                          ('ohe', OneHotEncoder(sparse = 'False', handle_unknown = 'ignore'))])

In [79]:
transformers = [('cat', cat_pipe, cat_columns), 
                ('num', num_pipe, num_columns), 
                ('num_bins', num_bins_pipe, num_columns_for_bins)]

In [80]:
ct = ColumnTransformer(transformers = transformers)

In [81]:
ml_pipe = Pipeline([('tf', ct),
                    ('logreg', LogisticRegression(solver = 'liblinear'))])

In [82]:
param_grid = {
    'logreg__C': [0.1, 0.2, 0.3],
    'logreg__solver': ['lbfgs', 'liblinear'],
    'tf__num_bins__bin__bins': [[-np.inf, 30, 40, 50, 60, np.inf], 
                             [-np.inf, 25, 35, 45, 55, 65, np.inf]]
}

In [83]:
import warnings
warnings.filterwarnings("ignore")

In [84]:
gs = GridSearchCV(ml_pipe, 
                  param_grid, 
                  scoring='roc_auc', 
                  cv=5, 
                  return_train_score=False)

# выполняем решетчатый поиск
gs.fit(X_train, y_train)

# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение AUC
print('Наилучшее значение AUC: {:.3f}'.format(gs.best_score_))
# смотрим значение AUC на тестовой выборке
print('AUC на тестовой выборке: {:.3f}'.format(
    roc_auc_score(y_test, gs.predict_proba(X_test)[:, 1])))

Наилучшие значения гиперпараметров: {'logreg__C': 0.1, 'logreg__solver': 'liblinear', 'tf__num_bins__bin__bins': [-inf, 25, 35, 45, 55, 65, inf]}
Наилучшее значение AUC: 0.687
AUC на тестовой выборке: 0.707


In [85]:
#метрика accuracy - значение scoring по умолчанию в GridSearchCV
gs = GridSearchCV(ml_pipe, 
                  param_grid, 
#                  scoring='roc_auc', 
                  cv=5, 
                  return_train_score=False)

# выполняем решетчатый поиск
gs.fit(X_train, y_train)

# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение accuracy
print('Наилучшее значение accuracy: {:.3f}'.format(gs.best_score_))
# смотрим значение accuracy на тестовой выборке
print('accuracy на тестовой выборке: {:.3f}'.format(gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'logreg__C': 0.1, 'logreg__solver': 'liblinear', 'tf__num_bins__bin__bins': [-inf, 30, 40, 50, 60, inf]}
Наилучшее значение accuracy: 0.880
accuracy на тестовой выборке: 0.879
